In [ ]:
# Local clone
! git clone https://github.com/nanopiero/CML_processing_by_ML.git

In [2]:
# Imports
from os.path import join, isdir, isfile
from os import listdir as ls
import copy
import torch
import numpy as np

import sys
sys.path.append('CML_processing_by_ML')
from src.utils.simulation import create_dataloader
import src.utils.architectures_fcn
from src.utils.architectures import load_archi
from src.utils.architectures_fcn import UNet_causal_5mn_atrous

In [3]:
# Dictionary with pseudo "distances" (distances between two antennnas) for 1,000 pseudo CML ids.
idx2distance = {i: 0.2 +  1.8 * torch.rand((1,)).item() for i in range(0, 1000)}
duration = 4096  # Duration for each tensor pair
batch_size = 100  # Number of samples per batch
dataloader = create_dataloader(duration, idx2distance, batch_size)

In [4]:
# Here we samples 64 ground-truth rainy processes and their noisy counterpart
# A rainy process is modeled by a 1-d Neymann-Scott process
# The Intensity of the Poisson process for parent events is 0.05 x distance
# The resulting rainy process is divided by the distance to give the "ground truth"
# while it is corrupted through the following steps to yield "noisy_series":
# - applying a non linear conversion to an attenuation in db
# - applying a "wet antenna convolution filter" (kind of sliding mean)
# - adding a high (gaussian noise with non linear dependance of sigma wrt the intensity)
# - adding a low frequency random processes

for batch_idx, (idxs, dists, ground_truths, noisy_series) in enumerate(dataloader):
  if batch_idx == 0:
    break

In [ ]:
import matplotlib.pyplot as plt
sigma = 2
for k in range(5):
  print(idxs[k], dists[k])
  plt.figure(figsize=(10, 6))
  plt.plot(np.arange(duration), ground_truths[k], label='ground_truth')
  plt.plot(np.arange(duration), noisy_series[k], label='predictor')
  plt.title(f'Inputs and Targets for the CML n°{idxs[k].item():.0f} (ditance: {dists[k].item():.2f})')
  plt.xlabel('Time (minutes)')
  plt.ylabel('Event Density')
  plt.ylim(-1,6)
  plt.legend()
  plt.show()

In [ ]:
arch = "UNet_causal_5mn_atrous"
nchannels = 1
nclasses = 1 # Regression only
dilation = 2
atrous_rates=[6, 12, 18] #, 24, 30, 36, 42]
additional_parameters = 0

model = load_archi(arch, nchannels, nclasses, size=64, dilation=1,
                   atrous_rates=atrous_rates, fixed_cumul=False,
                   additional_parameters=additional_parameters)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# model = UNet(1, 1, 16).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()

num_epochs = 50  # Adjust based on your needs

model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for batch_idx, (idxs, dists, ground_truths, noisy_series) in enumerate(dataloader):
        inputs, targets = noisy_series.to(device), \
                          ground_truths.to(device)

        # Add the channel's dim
        inputs = inputs.unsqueeze(1)
        targets = targets.unsqueeze(1)

        # Zeroing gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')

In [ ]:
import matplotlib.pyplot as plt

def visualize_predictions(model, data_loader, num_samples=1):
    model.eval()
    L = 1000
    with torch.no_grad():
        for batch_idx, (idxs, dists, ground_truths, noisy_series) in enumerate(dataloader):

            inputs = noisy_series.to(device).unsqueeze(1).float()  # Adjust input dimensions
            outputs = model(inputs).cpu()

            for i in range(num_samples):
                plt.figure(figsize=(16, 4))
                # plt.plot(noisy_series[i].squeeze(), label='input')
                plt.plot(ground_truths[i,:L].squeeze(), label='Observation')
                plt.plot(outputs[i].squeeze()[:L], label='Prediction', linestyle='--')
                plt.legend()
                plt.show()
            break  # Just show the first batch

visualize_predictions(model, dataloader, num_samples=5)


In [ ]:
# How to do better ?